# Make Features

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
%matplotlib inline
import pickle

#### Load Data

In [2]:
train = pd.read_csv(os.path.abspath("../dat")+'/train.csv', encoding='cp949')
test = pd.read_csv(os.path.abspath("../dat")+'/test.csv',  encoding='cp949')
data = pd.read_csv(os.path.abspath("../dat")+'/book_info.csv',  encoding='cp949')

In [3]:
# 결측값 확인

data.isnull().sum()

item_id       0
category      0
author       60
publisher     0
pub_date     70
dtype: int64

In [4]:
# book_info 의 pub_date의 오류로 보이는 데이터 변환 (인덱스 번호: 90867,493398)
data.loc[90867, 'pub_date'] = 201409
data.loc[49398, 'pub_date'] = 201505

### Feature Engineering

In [5]:
# 유의사항) Test Data에 있는 책 중에 1014개는 Train Data에는 없음.

len(list(set(test.item_id) - set(test.item_id).intersection(set(train.item_id)))), len(test)

(1014, 13481)

In [6]:
# 월별 도서별 판매량 형식으로 데이터 변환

df = train.groupby(['month','item_id'])['count'].agg([('item_cnt_month', 'sum')]).reset_index()

In [7]:
# target value(월별 도서 판매량)를 최대 20으로 조정

df['item_cnt_month'] = df['item_cnt_month'].clip(0,20)

In [8]:
# Add a book category feature

df = pd.merge(df, 
              train.drop_duplicates(subset='item_id').loc[:,['item_id', 'category']], 
              on='item_id', how='left')

#### Append test set

In [9]:
# 학습 데이터와 같은 feature를 만들기 위해 병합

test['month'] = 12  
df = pd.concat([df, test], ignore_index=True, sort=False).fillna(0)

In [10]:
# data의 feature를 df에 추가 (Add a book title, author, publisher, pub_date feature)

df = pd.merge(df, 
              data.drop_duplicates(subset='item_id').loc[:,['item_id', 'author', 'publisher', 'pub_date']], 
              on='item_id', how='left')

#### Add lag features

In [11]:
# 시차변수(1개월, 2개월, 3개월, 4개월 전 판매량) 생성

def lag_feature(df, lags, col):
    tmp = df[['month','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['month','item_id', col+'_lag_{}'.format(i)]
        shifted['month'] += i
        df = pd.merge(df, shifted, on=['month','item_id'], how='left')
    return df

In [12]:
df = lag_feature(df, [1,2,3,4], 'item_cnt_month')

In [13]:
# 도서별 월평균 판매량
# month_item_avg_item_cnt_lag_1,2,3,4

group = df.groupby(['month', 'item_id'])['item_cnt_month'].agg([('month_item_avg_item_cnt', 'mean')]).reset_index()
df = pd.merge(df, group, on=['month','item_id'], how='left')
df = lag_feature(df, [1,2,3,4], 'month_item_avg_item_cnt')
df.drop(['month_item_avg_item_cnt'], axis=1, inplace=True)

In [14]:
# 도서 분야별 월평균 판매량
# month_cat_avg_item_cnt_lag_1,2,3,4

group = df.groupby(['month', 'category'])['item_cnt_month'].agg([('month_cat_avg_item_cnt', 'mean')]).reset_index()
df = pd.merge(df, group, on=['month', 'category'], how='left')
df = lag_feature(df, [1,2,3,4], 'month_cat_avg_item_cnt')
df.drop(['month_cat_avg_item_cnt'], axis=1, inplace=True)

#### 추가 속성

In [15]:
# 월별 Count Lag => cat_delta_cnt_lag

group = df.groupby(['category']).agg({'item_cnt_month': ['mean']})
group.columns = ['category_avg_cnt_month']
group.reset_index(inplace=True)

df = pd.merge(df, group, on=['category'], how='left')
df['category_avg_cnt_month'] = df['category_avg_cnt_month'].astype(np.float16)

###
group = df.groupby(['month','category']).agg({'item_cnt_month': ['mean']})
group.columns = ['month_category_avg_cnt_month']
group.reset_index(inplace=True)

df = pd.merge(df, group, on=['month','category'], how='left')
df['month_category_avg_cnt_month'] = df['month_category_avg_cnt_month'].astype(np.float16)

###
lags = [1,2,3,4]
df = lag_feature(df, lags, 'month_category_avg_cnt_month')

for i in lags:
    df['delta_cnt_lag_'+str(i)] = \
        (df['month_category_avg_cnt_month_lag_'+str(i)] - df['category_avg_cnt_month']) / df['category_avg_cnt_month']
    
def select_trend(row):
    for i in lags:
        if row['delta_cnt_lag_'+str(i)]:
            return row['delta_cnt_lag_'+str(i)]
    return 0

# delta_cnt_lag 
df['cat_delta_cnt_lag'] = df.apply(select_trend, axis=1)
df['cat_delta_cnt_lag'] = df['cat_delta_cnt_lag'].astype(np.float16)
df['cat_delta_cnt_lag'].fillna(0, inplace=True)

fetures_to_drop = ['category_avg_cnt_month', 'month_category_avg_cnt_month']
for i in lags:
    fetures_to_drop += ['month_category_avg_cnt_month_lag_'+str(i)]
    fetures_to_drop += ['delta_cnt_lag_'+str(i)]

df.drop(fetures_to_drop, axis=1, inplace=True)

#### (추가 feature) 도서 작가별 월평균 판매량

In [16]:
# 도서 작가별 월평균 판매량
#month_aut_avg_item_cnt_lag_1,2,3,4

group = df.groupby(['month', 'author'])['item_cnt_month'].agg([('month_aut_avg_item_cnt', 'mean')]).reset_index()
df = pd.merge(df, group, on=['month', 'author'], how='left')
df = lag_feature(df, [1,2,3,4], 'month_aut_avg_item_cnt')
df.drop(['month_aut_avg_item_cnt'], axis=1, inplace=True)

#### (추가 feature) 도서 출판사별 월평균 판매량

In [17]:
# 도서 출판사별 월평균 판매량
#month_pub_avg_item_cnt_lag_1,2,3,4

group = df.groupby(['month', 'publisher'])['item_cnt_month'].agg([('month_pub_avg_item_cnt', 'mean')]).reset_index()
df = pd.merge(df, group, on=['month', 'publisher'], how='left')
df = lag_feature(df, [1,2,3,4], 'month_pub_avg_item_cnt')
df.drop(['month_pub_avg_item_cnt'], axis=1, inplace=True)

#### (추가 feature) 출간일(pub_date)별 => pub_date_year

In [18]:
# 출간일(pub_date)별=>pub_date_year
df['pub_date_year'] = df['pub_date'].astype('Int32') // 100

# NA는 중앙값으로 채움
df['pub_date_year'].fillna(df['pub_date_year'].median(), inplace=True)
df = df.drop(columns=['pub_date'])

#### 범주의 종류가 많은 feature 제거
=> 제거하지 않고 `JamesSteinEncoder`로 사용 고려 !!!

In [19]:
df = df.drop(columns=['author', 'publisher'])

#### Final data preparation

In [20]:
# Lag feature를 적용할 수 없는 1월부터 4월 데이터 제거

df = df[df.month > 4]

In [21]:
# 결측값 0으로 치환

def fill_na(df):
    df = df.replace([np.inf, -np.inf], np.nan)
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            if ('item_cnt' in col):
                df[col].fillna(0, inplace=True)
                
    return df

df = fill_na(df)

In [22]:
df

,month,item_id,item_cnt_month,category,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_4,month_item_avg_item_cnt_lag_1,month_item_avg_item_cnt_lag_2,...,cat_delta_cnt_lag,month_aut_avg_item_cnt_lag_1,month_aut_avg_item_cnt_lag_2,month_aut_avg_item_cnt_lag_3,month_aut_avg_item_cnt_lag_4,month_pub_avg_item_cnt_lag_1,month_pub_avg_item_cnt_lag_2,month_pub_avg_item_cnt_lag_3,month_pub_avg_item_cnt_lag_4,pub_date_year
83204,5,100000,1.0,잡지,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
83205,5,100004,1.0,전집,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2014
83206,5,100030,1.0,잡지,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
83207,5,100038,1.0,청소년,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2012
83208,5,100040,1.0,중고등학습서,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237802,12,203058,0.0,중고등학습서,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
237803,12,203061,0.0,IT 모바일,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
237804,12,203065,0.0,국어 외국어 사전,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
237805,12,203068,0.0,수험서 자격증,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014


# 주별 lag features와 병합 후 selection

In [23]:
df_2= pd.read_pickle(os.path.abspath("../dat")+'/features_week.pkl')

In [24]:
# features_3 에서 선택한 feature 만 선택
lis=[]
lis.append('item_id')
lis.append('month')
for i in range(1,7):
    for j in range(1,3):
        lis.append(f'week{i}_cat_avg_item_cnt_lag_{j}')
        lis.append(f'week{i}_item_avg_item_cnt_lag_{j}')
        

In [25]:
df_2[lis]

,item_id,month,week1_cat_avg_item_cnt_lag_1,week1_item_avg_item_cnt_lag_1,week1_cat_avg_item_cnt_lag_2,week1_item_avg_item_cnt_lag_2,week2_cat_avg_item_cnt_lag_1,week2_item_avg_item_cnt_lag_1,week2_cat_avg_item_cnt_lag_2,week2_item_avg_item_cnt_lag_2,...,week4_cat_avg_item_cnt_lag_2,week4_item_avg_item_cnt_lag_2,week5_cat_avg_item_cnt_lag_1,week5_item_avg_item_cnt_lag_1,week5_cat_avg_item_cnt_lag_2,week5_item_avg_item_cnt_lag_2,week6_cat_avg_item_cnt_lag_1,week6_item_avg_item_cnt_lag_1,week6_cat_avg_item_cnt_lag_2,week6_item_avg_item_cnt_lag_2
0,100000,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100004,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100030,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100038,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100040,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154598,203058,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
154599,203061,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
154600,203065,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
154601,203068,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
df = df.merge(df_2[lis],how='left')

In [ ]:
# pickle 형태로 분석에 사용할 최종 데이터 저장 및 시각화

df.to_pickle(os.path.abspath("../dat")+'/features_조기흠.pkl')

df_sampled = df.sample(n = 500).iloc[:,[2,4,5,6,7,8,9,10,11]]
sns.pairplot(df_sampled, diag_kind = 'kde')
plt.show()

# END